The following code will wrangle the original dataset from Yelp Data Challenge. Two original datasets were used business.json and review.json. The top popular categories for restaurants were selected for cities of Pittsburg, Phoenix, and Charlotte. The corresponding reviews for each restaurant business from review.json were joined with business.json by the business_id. 

In [7]:
#Business detail URL-- 'https://api.yelp.com/v3/businesses/{id}'
#Business reviews URL--'https://api.yelp.com/v3/businesses/{id}/reviews'

#Featured event URL -- 'https://api.yelp.com/v3/events/featured'
#Event search URL -- 'https://api.yelp.com/v3/events'
#Event look up URL -- 'https://api.yelp.com/v3/events/{id}'

import json
import pandas as pd



fileJson = 'yelp_dataset/business.json'

#load json file
dat = []
headers = True
with open(fileJson, encoding="cp866") as f:
     for line in f:
         json_object =json.loads(line)
         dat.append(json_object)
         if headers:
             keys =[]
             #Read the Head of the datasets
             for k, v in json_object.items():
                 keys.append(k)

dat = pd.read_json(fileJson, lines=True)



# List all the categories of food/restaurant in Pittsburg, Phoenix and Charlotte
business_food = dat[dat['categories'].str.contains('Restaurant', case=False, na=False)]
city_food = business_food[business_food['city'].str.contains('Pittsburg|Phoenix|Charlotte', case = False, na = False)]


#Create a data frame that gathers all the categories into a individual row
df_explode = city_food.assign(categories = city_food.categories
                         .str.split(', ')).explode('categories')
#List 50 most popular categories for restaurant records in Pittsburg, Phoenix and Charlotte
print(df_explode.categories.value_counts()[:50])
print(df_explode.head(10))

Restaurants                  9170
Food                         2421
Nightlife                    1462
Sandwiches                   1434
Bars                         1407
American (Traditional)       1374
Fast Food                    1308
American (New)               1115
Mexican                      1114
Pizza                        1099
Breakfast & Brunch            939
Burgers                       905
Italian                       697
Coffee & Tea                  554
Chinese                       545
Salad                         528
Event Planning & Services     501
Chicken Wings                 491
Seafood                       426
Delis                         420
Cafes                         396
Caterers                      364
Sports Bars                   313
Sushi Bars                    305
Bakeries                      288
Specialty Food                285
Japanese                      278
Mediterranean                 271
Desserts                      263
Barbeque      

According to the ouput from above, we can find some repetitive categories such as Food, which is relevant to restaurants. I only care about cateogries that describe the types of the food the restaurants server, instead of general categories of the restaurants. Some categories will be screened out.

In [15]:
df_filtered = df_explode[ (df_explode['categories'] != 'Restaurants') & (df_explode['categories'] != 'Food')
                       & (df_explode['categories'] != 'Nightlife') & (df_explode['categories'] != 'Bars')
                       & (df_explode['categories'] != 'Bars') & (df_explode['categories'] != 'Event Planning & Services')
                       & (df_explode['categories'] != 'Caterers') & (df_explode['categories'] != 'Sports Bars')
                       & (df_explode['categories'] != 'Wine Bars') & (df_explode['categories'] != 'Pubs')
                       & (df_explode['categories'] != 'Arts & Entertainment') & (df_explode['categories'] != 'Juice Bars & Smoothies')
                       & (df_explode['categories'] != 'Ice Cream & Frozen Yogurt') & (df_explode['categories'] != 'Cocktail Bars')
                       & (df_explode['categories'] != 'Wine & Spirits') & (df_explode['categories'] != 'Food Delivery Services')]

print(df_filtered.categories.value_counts()[:50])
print(df_filtered.head(10))

Sandwiches                1434
American (Traditional)    1374
Fast Food                 1308
American (New)            1115
Mexican                   1114
Pizza                     1099
Breakfast & Brunch         939
Burgers                    905
Italian                    697
Coffee & Tea               554
Chinese                    545
Salad                      528
Chicken Wings              491
Seafood                    426
Delis                      420
Cafes                      396
Sushi Bars                 305
Bakeries                   288
Specialty Food             285
Japanese                   278
Mediterranean              271
Desserts                   263
Barbeque                   263
Asian Fusion               241
Diners                     222
Food Trucks                211
Beer                       207
Vegetarian                 204
Steakhouses                194
Greek                      185
Southern                   184
Hot Dogs                   168
Tex-Mex 

To get the review of the bussiness from customer, the following step will connect the review.json file with the business file by the business ID.  

In [18]:
review_json= 'yelp_dataset/review.json'

#review is a dataset of 5G with millions of records, it is loaded by chunck instead of individual line
size = 1000000
review = pd.read_json(review_json, lines = True, dtype = {"review_id": str,
                                                         'user_id': str, 
                                                         'business_id': str,
                                                         'stars': int, 
                                                         'date': str, 'text':str,
                                                         'useful': int, 'funny': int,
                                                         'cool': int}, chunksize = size)


#Join review file and business file by using business_id.
#The merge was performed as the unit of chunk.
chunk_list = []
for review_chunk in review:
    #rename colume name to avoid conflict with business overall star rating
    chunk_merged = pd.merge(df_filtered, review_chunk, on = 'business_id', how = 'inner')
    #show feedback on progress 
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
#concatenate all relevant data back to one dataframe
restaurant_review = pd.concat(chunk_list, ignore_index = True, join = 'outer', axis = 0)

    
print(restaurant_review.head(10))

391338 out of 1,000,000 related reviews
367712 out of 1,000,000 related reviews
361638 out of 1,000,000 related reviews
359939 out of 1,000,000 related reviews
380255 out of 1,000,000 related reviews
348078 out of 1,000,000 related reviews
219536 out of 1,000,000 related reviews
              business_id                         name  \
0  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
1  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
2  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
3  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
4  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
5  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
6  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
7  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
8  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   
9  gnKjwL_1w79qoiV3IC_xQQ  Musashi Japanese Restaurant   

                     address       city state postal_code   latitude  \
0  10110 